In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!nvidia-smi

Mon Apr 15 15:50:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install diffusers
!pip install transformers scipy ftfy accelerate ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 782.3 kB/s eta 0:00:00a 0:00:01


In [3]:
!pip install torch torchvision torchaudio
!pip install bitsandbytes
!pip install peft
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.0 MB/s eta 0:00:00a 0:00:01m


In [4]:
import numpy as np
w_list = np.array([2,3,4,7])

import random
x_list = []
for _ in range(10):
    x_sample = np.array([random.randint(1,100) for _ in range(len(w_list))])
    x_list.append(x_sample)

y_list = []
for x_sample in x_list:
    y_temp = x_sample@w_list
    y_list.append(y_temp)

print("x_list:",x_list)
print("y_list:",y_list)

x_list: [array([21, 68, 64, 64]), array([95, 14, 53, 15]), array([ 5, 65, 51, 68]), array([20,  6, 75, 56]), array([82, 29, 92, 79]), array([74, 79, 76, 43]), array([56, 89, 41, 85]), array([90, 68, 26, 34]), array([90, 69, 33, 45]), array([97,  6, 23, 33])]
y_list: [950, 549, 885, 750, 1172, 990, 1138, 726, 834, 535]


In [5]:
import torch 
import torch.nn as nn

class MyLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.randn(4))

    def forward(self, x:torch.Tensor):
        return self.w @ x

In [6]:
model = MyLinear()

In [7]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001)

In [8]:
x_input = torch.tensor(x_list, dtype=torch.float32)
y_output = torch.tensor(y_list, dtype=torch.float32)

/tmp/ipykernel_34/198170575.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  x_input = torch.tensor(x_list, dtype=torch.float32)


In [9]:
# start train model
num_epochs = 100
for epoch in range(num_epochs):
    for i, x in enumerate(x_input):
        # forward
        y_pred = model(x)

        # calculate loss
        loss = loss_fn(y_pred,y_output[i])

        # zero out the cached parameter.
        optimizer.zero_grad()

        # backward
        loss.backward()

        # update paramters
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

print("train done") 

Epoch [10/100], Loss: 770.0049
Epoch [20/100], Loss: 428.5082
Epoch [30/100], Loss: 185.8403
Epoch [40/100], Loss: 78.7310
Epoch [50/100], Loss: 33.2101
Epoch [60/100], Loss: 13.9925
Epoch [70/100], Loss: 5.8939
Epoch [80/100], Loss: 2.4824
Epoch [90/100], Loss: 1.0456
Epoch [100/100], Loss: 0.4403
train done


In [10]:
model.w

Parameter containing:
tensor([1.9977, 3.0115, 4.0167, 6.9773], requires_grad=True)

In [11]:
from accelerate import utils
utils.write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [14]:
from accelerate import utils
utils.write_basic_config()

from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device

x_input.to(device)
y_output.to(device)
model.to(device)

model, optimizer = accelerator.prepare(
    model, optimizer
)

num_epochs = 100
for epoch in range(num_epochs):
    for i, x in enumerate(x_input):
        # forward
        y_pred = model(x)

        # calculate loss
        loss = loss_fn(y_pred,y_output[i])

        # zero out the cached parameter.
        optimizer.zero_grad()

        # backward
        #loss.backward()
        accelerator.backward(loss)

        # update paramters
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

print("train done") 

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensor in method wrapper_CUDA__dot)

In [13]:
accelerator.backward(loss)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [15]:
model = accelerator.unwrap_model(model)
model.w

Parameter containing:
tensor([1.9977, 3.0115, 4.0167, 6.9773], device='cuda:0', requires_grad=True)

In [19]:
import numpy as np
w_list = np.array([2,3,4,7])

import random
x_list = []
for _ in range(10):
    x_sample = np.array([random.randint(1,100) for _ in range(len(w_list))])
    x_list.append(x_sample)

y_list = []
for x_sample in x_list:
    y_temp = x_sample@w_list
    y_list.append(y_temp)
train_obj = {
    'w_list':w_list.tolist()
    , 'input':x_list
    , 'output':y_list
}

import pickle
with open('train_data.pkl','wb') as f:
    pickle.dump(train_obj,f)

In [20]:
import torch
import torch.nn as nn
from accelerate import utils
from accelerate import Accelerator

# start a accelerate instance
utils.write_basic_config()
accelerator = Accelerator()
device = accelerator.device

def main():
    # define the model
    class MyLinear(nn.Module):
        def __init__(self):
            super().__init__()
            self.w = nn.Parameter(torch.randn(len(w_list)))

        def forward(self, x:torch.Tensor):
            return self.w @ x

    # load training data
    import pickle
    with open("train_data.pkl",'rb') as f:
        loaded_object = pickle.load(f)
    w_list = loaded_object['w_list']
    x_list = loaded_object['input']
    y_list = loaded_object['output']

    # convert data to torch tensor
    x_input     = torch.tensor(x_list, dtype=torch.float32).to(device)
    y_output    = torch.tensor(y_list, dtype=torch.float32).to(device)

    # initialize model, loss function, and optimizer
    model       = MyLinear().to(device)
    loss_fn     = nn.MSELoss()
    optimizer   = torch.optim.SGD(model.parameters(), lr = 0.00001)

    # wrap model and optimizer using accelerate
    model, optimizer = accelerator.prepare(
        model, optimizer
    )

    num_epochs = 100
    for epoch in range(num_epochs):
        for i, x in enumerate(x_input):
            # forward
            y_pred = model(x)

            # calculate loss
            loss = loss_fn(y_pred,y_output[i])

            # zero out the cached parameter.
            optimizer.zero_grad()

            # backward
            #loss.backward()
            accelerator.backward(loss)

            # update paramters
            optimizer.step()

        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    # take a look at the model weights after trainning
    model = accelerator.unwrap_model(model)
    print(model.w)

if __name__ == "__main__":
    main()

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
Epoch [10/100], Loss: 94.4351
Epoch [20/100], Loss: 1.7403
Epoch [30/100], Loss: 0.0766
Epoch [40/100], Loss: 0.0737
Epoch [50/100], Loss: 0.0181
Epoch [60/100], Loss: 0.0032
Epoch [70/100], Loss: 0.0005
Epoch [80/100], Loss: 0.0001
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
Parameter containing:
tensor([2.0001, 2.9999, 4.0000, 7.0000], device='cuda:0', requires_grad=True)


In [21]:
# import packages 
import torch
from accelerate import utils
from accelerate import Accelerator
from diffusers import DDPMScheduler,StableDiffusionPipeline
from peft import LoraConfig
from peft.utils import get_peft_model_state_dict
from datasets import load_dataset
from torchvision import transforms
import math
from diffusers.optimization import get_scheduler
from tqdm.auto import tqdm
import torch.nn.functional as F
from diffusers.utils import convert_state_dict_to_diffusers

# train code 
def main():
    accelerator = Accelerator(
        gradient_accumulation_steps = gradient_accumulation_steps
        , mixed_precision           = "fp16" 
    )
    device      = accelerator.device
    ...
    # almost all training code will be land inside of this main function. 

if __name__ == "__main__":
    main()

2024-04-15 16:00:19.091285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 16:00:19.091433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 16:00:19.365642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


NameError: name 'gradient_accumulation_steps' is not defined

In [22]:
# hyperparameters
output_dir                      = "."
pretrained_model_name_or_path   = "runwayml/stable-diffusion-v1-5"
lora_rank                       = 4
lora_alpha                      = 4
learning_rate                   = 1e-4
adam_beta1, adam_beta2          = 0.9, 0.999
adam_weight_decay               = 1e-2
adam_epsilon                    = 1e-08
dataset_name                    = None     
train_data_dir                  = "./train_data"
top_rows                        = 4
output_dir                      = "output_dir"
resolution                      = 768
center_crop                     = True
random_flip                     = True
train_batch_size                = 4
gradient_accumulation_steps     = 1
num_train_epochs                = 200
# The scheduler type to use. Choose between ["linear", "cosine", "cosine_with_restarts", "polynomial","constant", "constant_with_warmup"]   
lr_scheduler_name               = "constant" #"cosine"#
max_grad_norm                   = 1.0
diffusion_scheduler             = DDPMScheduler 

In [23]:
noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")
weight_dtype = torch.float16
pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path, 
    torch_dtype = weight_dtype
).to(device)
tokenizer, text_encoder = pipe.tokenizer, pipe.text_encoder
vae, unet               = pipe.vae, pipe.unet

scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

safety_checker/model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [24]:
unet.requires_grad_(False)
vae.requires_grad_(False)
text_encoder.requires_grad_(False)

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [25]:
unet_lora_config = LoraConfig(
    r                     = lora_rank
    , lora_alpha          = lora_alpha
    , init_lora_weights   = "gaussian"
    , target_modules      = ["to_k", "to_q", "to_v", "to_out.0"]
)

In [26]:
# Add adapter and make sure the trainable params are in float32.
unet.add_adapter(unet_lora_config)
for param in unet.parameters():
    # only upcast trainable parameters (LoRA) into fp32
    if param.requires_grad:
        param.data = param.to(torch.float32)

In [27]:
if dataset_name:
    # Downloading and loading a dataset from the hub. data will be saved to ~/.cache/huggingface/datasets by default
    dataset = load_dataset(dataset_name)
else:
    dataset = load_dataset(
        "imagefolder"
        , data_dir = train_data_dir
    )

train_data = dataset["train"]
dataset["train"] = train_data.select(range(top_rows))

# Preprocessing the datasets. We need to tokenize inputs and targets.
dataset_columns = list(dataset["train"].features.keys())
image_column, caption_column = dataset_columns[0],dataset_columns[1]

EmptyDatasetError: The directory at /kaggle/working/train_data doesn't contain any data files